In [22]:
import numpy as np
import pandas as pd
import requests
import requests_cache
import time
from IPython.core.display import clear_output
import json
import glob
import os
import multiprocessing

requests_cache.install_cache()

# Using SeatGeek API

In [2]:
## Make first request
client_id = 'MTg4MDc2NjV8MTU3MDQ4OTQ1MC45MQ',
client_secret= '956e777ac06bd66c8d6157d380b4be02d9c07a215c6f4694304b28f7ac2a4d6d'

headers = {
    'Content-Type':'application/x-www-form-urlencoded',
}

payload = {
    'client_id': client_id,
    'client_secret': client_secret,
    'format': 'json',
    'sort': 'datetime_utc.asc'
}

r = requests.get('https://api.seatgeek.com/2/', headers=headers, params=payload)
r.status_code

200

In [3]:
print(r)
print(r.text)

<Response [200]>
{"client_id": "MTg4MDc2NjV8MTU3MDQ4OTQ1MC45MQ", "message": "ok", "service": "api", "status": 200, "version": "2.0.0"}


In [7]:
# getting a response from an event id
event_response = requests.get("https://api.seatgeek.com/2/events/721901?client_id=MTg4MDc2NjV8MTU3MDQ4OTQ1MC45MQ&client_secret=956e777ac06bd66c8d6157d380b4be02d9c07a215c6f4694304b28f7ac2a4d6d")
event_response.json();

In [8]:
# getting a response from an event id; testing headers and params
event_response2 = requests.get("https://api.seatgeek.com/2/events/721901", headers=headers, params=payload)
event_response2.json();

In [9]:
# getting a response from an event id
performer_response = requests.get("https://api.seatgeek.com/2/performers/8741", headers=headers, params=payload)
performer_response.json();

In [4]:
event_onward_response = requests.get("https://api.seatgeek.com/2/events?taxonomies.name=concert", headers=headers, params=payload)
event_onward_response.json();

In [5]:
event_onward_response.json()['meta']

{'geolocation': None, 'per_page': 10, 'total': 31501, 'took': 6, 'page': 1}

In [ ]:
event_onward_response = requests.get("https://api.seatgeek.com/2/events?taxonomies.name=concert", headers=headers, params=payload)
event_onward_response.json();

In [6]:
event_onward_response.json()['events']

[{'links': [],
  'event_promotion': None,
  'conditional': False,
  'is_open': False,
  'id': 4626804,
  'stats': {'visible_listing_count': 3,
   'dq_bucket_counts': [0, 0, 3, 0, 0, 0, 0, 0],
   'average_price': 227,
   'lowest_sg_base_price_good_deals': None,
   'lowest_price_good_deals': None,
   'median_price': 227,
   'listing_count': 3,
   'variants': None,
   'lowest_price': 227,
   'lowest_sg_base_price': 177,
   'highest_price': 227},
  'title': 'Carolina Opry',
  'announce_date': '2018-11-08T00:00:00',
  'score': 0,
  'access_method': None,
  'announcements': {},
  'taxonomies': [{'name': 'concert', 'parent_id': None, 'id': 2000000}],
  'type': 'concert',
  'status': 'normal',
  'description': '',
  'datetime_local': '2019-10-09T19:00:00',
  'visible_until_utc': '2019-10-10T03:00:00',
  'time_tbd': False,
  'date_tbd': False,
  'performers': [{'image': 'https://seatgeek.com/images/performers-landscape/carolina-opry-b37aa4/58455/huge.jpg',
    'primary': True,
    'colors': Non

In [7]:
# Events from jan 1, 2014 onward
date_response = requests.get('https://api.seatgeek.com/2/events?datetime_utc.gt=2014-01-01', headers=headers, params=payload)

In [12]:
date_response.json()

{'meta': {'geolocation': None,
  'per_page': 10,
  'total': 122433,
  'took': 13,
  'page': 1},
 'events': [{'links': [],
   'event_promotion': None,
   'conditional': False,
   'is_open': False,
   'id': 4960704,
   'stats': {'visible_listing_count': None,
    'dq_bucket_counts': None,
    'average_price': None,
    'lowest_sg_base_price_good_deals': None,
    'lowest_price_good_deals': None,
    'median_price': None,
    'listing_count': None,
    'variants': None,
    'lowest_price': None,
    'lowest_sg_base_price': None,
    'highest_price': None},
   'title': 'TBD at San Francisco Giants: NLDS (Home Game 3, If Necessary)',
   'announce_date': '2019-08-13T00:00:00',
   'score': 0.704,
   'access_method': {'employee_only': False,
    'created_at': '2015-11-04T18:42:51Z',
    'method': 'QRCODE_TM'},
   'announcements': {},
   'taxonomies': [{'name': 'sports', 'parent_id': None, 'id': 1000000},
    {'name': 'baseball', 'parent_id': 1000000, 'id': 1010000},
    {'name': 'mlb', 'parent

In [8]:
date_response.json()['meta']

{'geolocation': None, 'per_page': 10, 'total': 122433, 'took': 13, 'page': 1}

In [9]:
date_response.json()['meta']['page']

1

In [10]:
len(date_response.json()['events'])

10

In [26]:
# initialize list for results
results = []

In [50]:
import os.path
import multiprocessing

In [79]:
import json

# # set intial page and a high total number
# total_pages = 122433
# page = 1

results = list(range(1,30096))

def runTask(arg):
    try:
        page = arg
        headers = {
        'Content-Type':'application/x-www-form-urlencoded',
        }

        payload = {
            'client_id': client_id,
            'client_secret': client_secret,
            'format': 'json',
            'page': page,
            'sort': 'datetime_utc.asc'
        }

        r = requests.get('https://api.seatgeek.com/2/events?taxonomies.name=concert', headers=headers, params=payload)
        print(r)
        # if we get an error, print the response and halt the loop
        if r.status_code != 200:
            print("Error on page {}".format(page))
            print(r.text)
            sys.exit()

        with open(f'{page}.json', 'w') as outfile:
            json.dump(r.json(), outfile)

        # if it's not a cached result, sleep
        if not getattr(r, 'from_cache', False):
            time.sleep(0.25)

    except Exception:
        print('here')
        pass

In [81]:
with multiprocessing.Pool(100) as workers:
        workers.map(runTask, results)

OSError: [Errno 24] Too many open files

# Reading in JSON files

In [75]:
with open('1.json') as json_file:
    json_data = json.load(json_file)

In [78]:
json_data

{'meta': {'geolocation': None,
  'per_page': 10,
  'total': 30096,
  'took': 6,
  'page': 1},
 'events': [{'links': [],
   'event_promotion': None,
   'conditional': False,
   'is_open': False,
   'id': 4668292,
   'stats': {'visible_listing_count': 4,
    'dq_bucket_counts': [0, 0, 0, 0, 0, 0, 0, 0],
    'average_price': 245,
    'lowest_sg_base_price_good_deals': None,
    'lowest_price_good_deals': None,
    'median_price': 231,
    'listing_count': 8,
    'variants': None,
    'lowest_price': 211,
    'lowest_sg_base_price': 164,
    'highest_price': 280},
   'title': 'The NeverEnding Story',
   'announce_date': '2018-12-15T00:00:00',
   'score': 0,
   'access_method': None,
   'announcements': {},
   'taxonomies': [{'name': 'concert', 'parent_id': None, 'id': 2000000}],
   'type': 'concert',
   'status': 'normal',
   'description': '',
   'datetime_local': '2019-10-17T12:30:00',
   'visible_until_utc': '2019-10-17T20:30:00',
   'time_tbd': False,
   'date_tbd': False,
   'performe

In [77]:
json_data['events'][0]

{'links': [],
 'event_promotion': None,
 'conditional': False,
 'is_open': False,
 'id': 4668292,
 'stats': {'visible_listing_count': 4,
  'dq_bucket_counts': [0, 0, 0, 0, 0, 0, 0, 0],
  'average_price': 245,
  'lowest_sg_base_price_good_deals': None,
  'lowest_price_good_deals': None,
  'median_price': 231,
  'listing_count': 8,
  'variants': None,
  'lowest_price': 211,
  'lowest_sg_base_price': 164,
  'highest_price': 280},
 'title': 'The NeverEnding Story',
 'announce_date': '2018-12-15T00:00:00',
 'score': 0,
 'access_method': None,
 'announcements': {},
 'taxonomies': [{'name': 'concert', 'parent_id': None, 'id': 2000000}],
 'type': 'concert',
 'status': 'normal',
 'description': '',
 'datetime_local': '2019-10-17T12:30:00',
 'visible_until_utc': '2019-10-17T20:30:00',
 'time_tbd': False,
 'date_tbd': False,
 'performers': [{'image': 'https://seatgeek.com/images/performers-landscape/the-neverending-story-6913f1/20288/huge.jpg',
   'primary': True,
   'colors': None,
   'images': 

In [7]:
json_data['events'][0]['stats']

{'visible_listing_count': 16,
 'dq_bucket_counts': [0, 0, 0, 0, 0, 0, 0, 0],
 'average_price': 56,
 'lowest_sg_base_price_good_deals': None,
 'lowest_price_good_deals': None,
 'median_price': 56,
 'listing_count': 32,
 'variants': None,
 'lowest_price': 49,
 'lowest_sg_base_price': 35,
 'highest_price': 63}

In [8]:
json_data['events'][0]['title']

'Eiffel Tower Experience - Las Vegas'

In [9]:
json_data['events'][0]['stats']['average_price']
json_data['events'][0]['stats']['lowest_price']
json_data['events'][0]['stats']['highest_price']
json_data['events'][0]['stats']['lowest_sg_base_price']
json_data['events'][0]['stats']['median_price']

56

In [10]:
json_data['events'][0]['venue']['postal_code']

'89109'

In [11]:
json_data['events'][0]['venue']['city']

'Las Vegas'

In [12]:
json_data['events'][0]['type']

'family'

In [13]:
json_data['events'][0]['popularity']

0.324

In [14]:
json_data['events'][0]['taxonomies']

[{'name': 'theater', 'parent_id': None, 'id': 3000000},
 {'name': 'family', 'parent_id': 3000000, 'id': 3050000}]

In [15]:
json_data['events'][0]['venue']

{'links': [],
 'metro_code': 839,
 'postal_code': '89109',
 'timezone': 'America/Los_Angeles',
 'has_upcoming_events': True,
 'id': 1610,
 'city': 'Las Vegas',
 'extended_address': 'Las Vegas, NV 89109',
 'display_location': 'Las Vegas, NV',
 'state': 'NV',
 'score': 0.45388848,
 'location': {'lat': 36.1125, 'lon': -115.171},
 'access_method': None,
 'num_upcoming_events': 161,
 'address': '3655 Las Vegas Boulevard South',
 'capacity': 1200,
 'slug': 'le-theatre-des-arts-at-paris-las-vegas',
 'name': 'Le Theatre Des Arts at Paris Las Vegas',
 'url': 'https://seatgeek.com/venues/le-theatre-des-arts-at-paris-las-vegas/tickets',
 'country': 'US',
 'popularity': 0,
 'name_v2': 'Le Theatre Des Arts at Paris Las Vegas'}

In [18]:
json_data['events'][0]['performers']

[{'image': 'https://seatgeek.com/images/performers-landscape/eiffel-tower-experience-4bb179/605657/huge.jpg',
  'primary': True,
  'colors': None,
  'images': {'huge': 'https://seatgeek.com/images/performers-landscape/eiffel-tower-experience-4bb179/605657/huge.jpg'},
  'has_upcoming_events': True,
  'id': 605657,
  'stats': {'event_count': 158},
  'image_license': 'https://creativecommons.org/licenses/by-nd/2.0/',
  'score': 0.39,
  'location': None,
  'taxonomies': [{'name': 'theater',
    'parent_id': None,
    'id': 3000000,
    'document_source': {'source_type': 'ELASTIC', 'generation_type': 'FULL'}},
   {'name': 'family',
    'parent_id': 3000000,
    'id': 3050000,
    'document_source': {'source_type': 'ELASTIC', 'generation_type': 'FULL'}}],
  'type': 'theater_family',
  'num_upcoming_events': 158,
  'short_name': 'Eiffel Tower Experience',
  'home_venue_id': None,
  'slug': 'eiffel-tower-experience',
  'divisions': None,
  'name': 'Eiffel Tower Experience',
  'url': 'https://s

In [20]:
pd.DataFrame.from_dict(json_data['events'][0]['performers'])

,colors,divisions,has_upcoming_events,home_venue_id,id,image,image_attribution,image_license,images,location,...,num_upcoming_events,popularity,primary,score,short_name,slug,stats,taxonomies,type,url
0,None,None,True,None,605657,https://seatgeek.com/images/performers-landsca...,https://www.flickr.com/photos/villageundergrou...,https://creativecommons.org/licenses/by-nd/2.0/,{'huge': 'https://seatgeek.com/images/performe...,None,...,158,0,True,0.39,Eiffel Tower Experience,eiffel-tower-experience,{'event_count': 158},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_family,https://seatgeek.com/eiffel-tower-experience-t...


In [109]:
df1 = pd.DataFrame.from_dict(json_data['events'])

In [99]:
json_data;

In [100]:
pd.DataFrame.from_dict(json_data['events']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 29 columns):
access_method        1 non-null object
announce_date        10 non-null object
announcements        10 non-null object
conditional          10 non-null bool
created_at           10 non-null object
date_tbd             10 non-null bool
datetime_local       10 non-null object
datetime_tbd         10 non-null bool
datetime_utc         10 non-null object
description          10 non-null object
enddatetime_utc      0 non-null object
event_promotion      0 non-null object
general_admission    2 non-null object
id                   10 non-null int64
is_open              10 non-null bool
links                10 non-null object
performers           10 non-null object
popularity           10 non-null float64
score                10 non-null float64
short_title          10 non-null object
stats                10 non-null object
status               10 non-null object
taxonomies           10 non-n

In [101]:
json_data['events'][0]['stats']

{'visible_listing_count': 16,
 'dq_bucket_counts': [0, 0, 0, 0, 0, 0, 0, 0],
 'average_price': 56,
 'lowest_sg_base_price_good_deals': None,
 'lowest_price_good_deals': None,
 'median_price': 56,
 'listing_count': 32,
 'variants': None,
 'lowest_price': 49,
 'lowest_sg_base_price': 35,
 'highest_price': 63}

In [102]:
json_data['events'][0]['performers']

[{'image': 'https://seatgeek.com/images/performers-landscape/eiffel-tower-experience-4bb179/605657/huge.jpg',
  'primary': True,
  'colors': None,
  'images': {'huge': 'https://seatgeek.com/images/performers-landscape/eiffel-tower-experience-4bb179/605657/huge.jpg'},
  'has_upcoming_events': True,
  'id': 605657,
  'stats': {'event_count': 158},
  'image_license': 'https://creativecommons.org/licenses/by-nd/2.0/',
  'score': 0.39,
  'location': None,
  'taxonomies': [{'name': 'theater',
    'parent_id': None,
    'id': 3000000,
    'document_source': {'source_type': 'ELASTIC', 'generation_type': 'FULL'}},
   {'name': 'family',
    'parent_id': 3000000,
    'id': 3050000,
    'document_source': {'source_type': 'ELASTIC', 'generation_type': 'FULL'}}],
  'type': 'theater_family',
  'num_upcoming_events': 158,
  'short_name': 'Eiffel Tower Experience',
  'home_venue_id': None,
  'slug': 'eiffel-tower-experience',
  'divisions': None,
  'name': 'Eiffel Tower Experience',
  'url': 'https://s

In [103]:
json_data['events'][0]['venue']

{'links': [],
 'metro_code': 839,
 'postal_code': '89109',
 'timezone': 'America/Los_Angeles',
 'has_upcoming_events': True,
 'id': 1610,
 'city': 'Las Vegas',
 'extended_address': 'Las Vegas, NV 89109',
 'display_location': 'Las Vegas, NV',
 'state': 'NV',
 'score': 0.45388848,
 'location': {'lat': 36.1125, 'lon': -115.171},
 'access_method': None,
 'num_upcoming_events': 161,
 'address': '3655 Las Vegas Boulevard South',
 'capacity': 1200,
 'slug': 'le-theatre-des-arts-at-paris-las-vegas',
 'name': 'Le Theatre Des Arts at Paris Las Vegas',
 'url': 'https://seatgeek.com/venues/le-theatre-des-arts-at-paris-las-vegas/tickets',
 'country': 'US',
 'popularity': 0,
 'name_v2': 'Le Theatre Des Arts at Paris Las Vegas'}

In [110]:
df1 = df1.assign(**pd.DataFrame(df1.stats.values.tolist()).add_prefix('stats_'))

In [111]:
df1 = df1.assign(**pd.DataFrame(df1.venue.values.tolist()).add_prefix('venue_'))

In [43]:
# df1 = df1.assign(**pd.DataFrame(df1.performers.values.tolist()).add_prefix('perfomers_'))

In [65]:
# test = pd.DataFrame(df1.performers.values.tolist()).add_prefix('performers_')
# test

,performers_0,performers_1
0,{'image': 'https://seatgeek.com/images/perform...,None
1,"{'image': None, 'primary': True, 'colors': Non...",None
2,{'image': 'https://seatgeek.com/images/perform...,"{'image': None, 'primary': True, 'colors': Non..."
3,{'image': 'https://seatgeek.com/images/perform...,{'image': 'https://seatgeek.com/images/perform...
4,{'image': 'https://seatgeek.com/images/perform...,None
5,{'image': 'https://seatgeek.com/images/perform...,None
6,{'image': 'https://seatgeek.com/images/perform...,None
7,{'image': 'https://seatgeek.com/images/perform...,{'image': 'https://seatgeek.com/images/perform...
8,{'image': 'https://seatgeek.com/images/perform...,None
9,"{'image': None, 'primary': True, 'colors': Non...",None


In [66]:
# pd.DataFrame(test.performers_0.values.tolist()).add_prefix('perfomers_0_')

,perfomers_0_colors,perfomers_0_divisions,perfomers_0_genres,perfomers_0_has_upcoming_events,perfomers_0_home_venue_id,perfomers_0_id,perfomers_0_image,perfomers_0_image_attribution,perfomers_0_image_license,perfomers_0_images,...,perfomers_0_num_upcoming_events,perfomers_0_popularity,perfomers_0_primary,perfomers_0_score,perfomers_0_short_name,perfomers_0_slug,perfomers_0_stats,perfomers_0_taxonomies,perfomers_0_type,perfomers_0_url
0,None,None,NaN,True,None,605657,https://seatgeek.com/images/performers-landsca...,https://www.flickr.com/photos/villageundergrou...,https://creativecommons.org/licenses/by-nd/2.0/,{'huge': 'https://seatgeek.com/images/performe...,...,158,0,True,0.39,Eiffel Tower Experience,eiffel-tower-experience,{'event_count': 158},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_family,https://seatgeek.com/eiffel-tower-experience-t...
1,None,None,NaN,True,None,607050,None,None,None,{},...,158,0,True,0.00,High Roller Wheel,high-roller-wheel,{'event_count': 158},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_family,https://seatgeek.com/high-roller-wheel-tickets
2,None,None,NaN,True,None,6695,https://seatgeek.com/images/performers-landsca...,None,None,{'huge': 'https://seatgeek.com/images/performe...,...,45,0,NaN,0.44,Swan Lake,swan-lake,{'event_count': 45},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_dance_performance_tour,https://seatgeek.com/swan-lake-tickets
3,None,None,"[{'name': 'Pop', 'image': 'https://seatgeek.co...",True,None,4313,https://seatgeek.com/images/performers-landsca...,Nigel Crane / Redferns / Getty Images,rightsmanaged,{'huge': 'https://seatgeek.com/images/performe...,...,15,0,True,0.45,Angels & Airwaves,angels-airwaves,{'event_count': 15},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/angels-airwaves-tickets
4,None,None,NaN,True,None,326247,https://seatgeek.com/images/performers-landsca...,https://www.flickr.com/photos/visitelpaso/1431...,https://creativecommons.org/licenses/by/2.0/,{'huge': 'https://seatgeek.com/images/performe...,...,3,0,True,0.43,The Aquadolls,the-aquadolls,{'event_count': 3},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/the-aquadolls-tickets
5,None,None,"[{'name': 'Pop', 'primary': True, 'id': 452, '...",True,None,1605,https://seatgeek.com/images/performers-landsca...,https://www.flickr.com/photos/nickperrone/8609...,https://creativecommons.org/licenses/by/2.0/,{'huge': 'https://seatgeek.com/images/performe...,...,13,0,True,0.43,Senses Fail,senses-fail,{'event_count': 13},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/senses-fail-tickets
6,None,None,"[{'name': 'Pop', 'image': 'https://seatgeek.co...",True,None,63653,https://seatgeek.com/images/performers-landsca...,Rick Kern / WireImage / Getty Images,rightsmanaged,{'huge': 'https://seatgeek.com/images/performe...,...,2,0,True,0.52,William Tyler,william-tyler,{'event_count': 2},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/william-tyler-tickets
7,None,None,"[{'name': 'Pop', 'primary': False, 'id': 452, ...",True,None,2711,https://seatgeek.com/images/performers-landsca...,Timothy Hiatt / WireImage / Getty Images,rightsmanaged,{'huge': 'https://seatgeek.com/images/performe...,...,24,0,True,0.60,Vampire Weekend,vampire-weekend,{'event_count': 24},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/vampire-weekend-tickets
8,None,None,NaN,True,None,639970,https://seatgeek.com/images/performers-landsca...,https://www.flickr.com/photos/villageundergrou...,https://creativecommons.org/licenses/by-nd/2.0/,{'huge': 'https://seatgeek.com/images/performe...,...,13,0,True,0.47,Bedouine,bedouine,{'event_count': 13},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/bedouine-tickets
9,None,None,NaN,True,None,724650,None,None,None,{},...,2,0,True,0.00,A Class Act,a-class-act,{'event_count': 2},"[{'name': 'theater', 'parent_

In [67]:
# pd.DataFrame(test.performers_1.values.tolist()).add_prefix('perfomers_1_')

,perfomers_1_0
0,None
1,None
2,"{'image': None, 'primary': True, 'colors': Non..."
3,{'image': 'https://seatgeek.com/images/perform...
4,None
5,None
6,None
7,{'image': 'https://seatgeek.com/images/perform...
8,None
9,None


In [88]:
# pd.DataFrame(df1.performers.str[0].tolist()).add_prefix('performers_')

,performers_colors,performers_divisions,performers_genres,performers_has_upcoming_events,performers_home_venue_id,performers_id,performers_image,performers_image_attribution,performers_image_license,performers_images,...,performers_num_upcoming_events,performers_popularity,performers_primary,performers_score,performers_short_name,performers_slug,performers_stats,performers_taxonomies,performers_type,performers_url
0,None,None,NaN,True,None,605657,https://seatgeek.com/images/performers-landsca...,https://www.flickr.com/photos/villageundergrou...,https://creativecommons.org/licenses/by-nd/2.0/,{'huge': 'https://seatgeek.com/images/performe...,...,158,0,True,0.39,Eiffel Tower Experience,eiffel-tower-experience,{'event_count': 158},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_family,https://seatgeek.com/eiffel-tower-experience-t...
1,None,None,NaN,True,None,607050,None,None,None,{},...,158,0,True,0.00,High Roller Wheel,high-roller-wheel,{'event_count': 158},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_family,https://seatgeek.com/high-roller-wheel-tickets
2,None,None,NaN,True,None,6695,https://seatgeek.com/images/performers-landsca...,None,None,{'huge': 'https://seatgeek.com/images/performe...,...,45,0,NaN,0.44,Swan Lake,swan-lake,{'event_count': 45},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_dance_performance_tour,https://seatgeek.com/swan-lake-tickets
3,None,None,"[{'name': 'Pop', 'image': 'https://seatgeek.co...",True,None,4313,https://seatgeek.com/images/performers-landsca...,Nigel Crane / Redferns / Getty Images,rightsmanaged,{'huge': 'https://seatgeek.com/images/performe...,...,15,0,True,0.45,Angels & Airwaves,angels-airwaves,{'event_count': 15},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/angels-airwaves-tickets
4,None,None,NaN,True,None,326247,https://seatgeek.com/images/performers-landsca...,https://www.flickr.com/photos/visitelpaso/1431...,https://creativecommons.org/licenses/by/2.0/,{'huge': 'https://seatgeek.com/images/performe...,...,3,0,True,0.43,The Aquadolls,the-aquadolls,{'event_count': 3},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/the-aquadolls-tickets
5,None,None,"[{'name': 'Pop', 'primary': True, 'id': 452, '...",True,None,1605,https://seatgeek.com/images/performers-landsca...,https://www.flickr.com/photos/nickperrone/8609...,https://creativecommons.org/licenses/by/2.0/,{'huge': 'https://seatgeek.com/images/performe...,...,13,0,True,0.43,Senses Fail,senses-fail,{'event_count': 13},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/senses-fail-tickets
6,None,None,"[{'name': 'Pop', 'image': 'https://seatgeek.co...",True,None,63653,https://seatgeek.com/images/performers-landsca...,Rick Kern / WireImage / Getty Images,rightsmanaged,{'huge': 'https://seatgeek.com/images/performe...,...,2,0,True,0.52,William Tyler,william-tyler,{'event_count': 2},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/william-tyler-tickets
7,None,None,"[{'name': 'Pop', 'primary': False, 'id': 452, ...",True,None,2711,https://seatgeek.com/images/performers-landsca...,Timothy Hiatt / WireImage / Getty Images,rightsmanaged,{'huge': 'https://seatgeek.com/images/performe...,...,24,0,True,0.60,Vampire Weekend,vampire-weekend,{'event_count': 24},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/vampire-weekend-tickets
8,None,None,NaN,True,None,639970,https://seatgeek.com/images/performers-landsca...,https://www.flickr.com/photos/villageundergrou...,https://creativecommons.org/licenses/by-nd/2.0/,{'huge': 'https://seatgeek.com/images/performe...,...,13,0,True,0.47,Bedouine,bedouine,{'event_count': 13},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/bedouine-tickets
9,None,None,NaN,True,None,724650,None,None,None,{},...,2,0,True,0.00,A Class Act,a-class-act,{'event_count': 2},"[{'name': 'theater', 'parent_id': None, 'id': ...

In [108]:
# df1.performers.str[0].toli?st();

AttributeError: 'DataFrame' object has no attribute 'performers'

In [113]:
df1 = df1.assign(**pd.DataFrame(df1.performers.str[0].tolist()).add_prefix('performers_'))

In [114]:
df1

,access_method,announce_date,announcements,conditional,created_at,date_tbd,datetime_local,datetime_tbd,datetime_utc,description,...,performers_num_upcoming_events,performers_popularity,performers_primary,performers_score,performers_short_name,performers_slug,performers_stats,performers_taxonomies,performers_type,performers_url
0,None,2019-01-23T00:00:00,{'checkout_disclosures': {'messages': [{'text'...,False,2019-01-23T16:10:16,False,2019-10-08T03:30:00,False,2019-10-08T10:30:00,,...,158,0,True,0.39,Eiffel Tower Experience,eiffel-tower-experience,{'event_count': 158},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_family,https://seatgeek.com/eiffel-tower-experience-t...
1,None,2018-12-20T00:00:00,{'checkout_disclosures': {'messages': [{'text'...,False,2018-12-20T16:11:07,False,2019-10-08T03:30:00,False,2019-10-08T10:30:00,,...,158,0,True,0.00,High Roller Wheel,high-roller-wheel,{'event_count': 158},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_family,https://seatgeek.com/high-roller-wheel-tickets
2,None,2019-05-02T00:00:00,{},False,2019-05-02T16:16:35,False,2019-10-08T19:30:00,False,2019-10-09T01:30:00,,...,45,0,NaN,0.44,Swan Lake,swan-lake,{'event_count': 45},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_dance_performance_tour,https://seatgeek.com/swan-lake-tickets
3,None,2019-05-14T00:00:00,{},False,2019-05-14T02:59:07,False,2019-10-08T18:30:00,False,2019-10-09T01:30:00,,...,15,0,True,0.45,Angels & Airwaves,angels-airwaves,{'event_count': 15},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/angels-airwaves-tickets
4,None,2019-08-24T00:00:00,{},False,2019-08-24T16:02:30,False,2019-10-08T20:30:00,False,2019-10-09T01:30:00,,...,3,0,True,0.43,The Aquadolls,the-aquadolls,{'event_count': 3},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/the-aquadolls-tickets
5,None,2019-07-13T00:00:00,{},False,2019-07-13T16:04:06,False,2019-10-08T19:30:00,False,2019-10-09T01:30:00,,...,13,0,True,0.43,Senses Fail,senses-fail,{'event_count': 13},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/senses-fail-tickets
6,None,2019-09-09T00:00:00,{},False,2019-09-09T16:04:28,False,2019-10-08T20:30:00,False,2019-10-09T01:30:00,,...,2,0,True,0.52,William Tyler,william-tyler,{'event_count': 2},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/william-tyler-tickets
7,"{'employee_only': False, 'created_at': '2015-1...",2019-01-31T00:00:00,{},False,2019-01-31T16:17:36,False,2019-10-08T19:30:00,False,2019-10-09T01:30:00,,...,24,0,True,0.60,Vampire Weekend,vampire-weekend,{'event_count': 24},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/vampire-weekend-tickets
8,None,2019-04-26T00:00:00,{},False,2019-04-26T16:21:18,False,2019-10-08T21:30:00,False,2019-10-09T01:30:00,,...,13,0,True,0.47,Bedouine,bedouine,{'event_count': 13},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/bedouine-tickets
9,None,2019-09-13T00:00:00,{},False,2019-09-13T16:08:37,False,2019-10-08T20:30:00,False,2019-10-09T01:30:00,,...,2,0,True,0.00,A Class Act,a-class-act,{'event_count': 2},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater,https://seatgeek.com/a-class-act-tickets


In [115]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 84 columns):
access_method                            1 non-null object
announce_date                            10 non-null object
announcements                            10 non-null object
conditional                              10 non-null bool
created_at                               10 non-null object
date_tbd                                 10 non-null bool
datetime_local                           10 non-null object
datetime_tbd                             10 non-null bool
datetime_utc                             10 non-null object
description                              10 non-null object
enddatetime_utc                          0 non-null object
event_promotion                          0 non-null object
general_admission                        2 non-null object
id                                       10 non-null int64
is_open                                  10 non-null bool
links      

In [71]:
result = pd.DataFrame()

def format_ticket_json(file):
    # read in json file
    with open(file) as json_file:
        json_data = json.load(json_file)
    
    # json dict to pandas pd
    df = pd.DataFrame.from_dict(json_data['events'])
    
    if set(['stats', 'venue', 'performers', 'taxonomies']).issubset(df.columns):
        # split the stats, venur, and performers columns
        df = df.assign(**pd.DataFrame(df.stats.values.tolist()).add_prefix('stats_'))
        df = df.assign(**pd.DataFrame(df.venue.values.tolist()).add_prefix('venue_'))
        df = df.assign(**pd.DataFrame(df.performers.str[0].tolist()).add_prefix('performers_'))
        df = df.assign(**pd.DataFrame(df.taxonomies.str[0].tolist()).add_prefix('taxonomies_'))
        return df
    else: 
        print("{} doesn't have appropriate columns".format(file))
           

In [72]:
result = result.append(format_ticket_json('2.json'))

In [73]:
result

,access_method,announce_date,announcements,conditional,created_at,date_tbd,datetime_local,datetime_tbd,datetime_utc,description,...,performers_score,performers_short_name,performers_slug,performers_stats,performers_taxonomies,performers_type,performers_url,taxonomies_id,taxonomies_name,taxonomies_parent_id
0,None,2019-10-03T00:00:00,{},False,2019-10-03T16:09:57,False,2019-10-08T20:30:00,False,2019-10-09T01:30:00,,...,0.00,Therapeutic Laughter Comedy Show[Hecklers Edit...,therapeutic-laughter-comedy-show-hecklers-edition,{'event_count': 1},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_comedy,https://seatgeek.com/therapeutic-laughter-come...,3000000,theater,None
1,None,2019-07-03T00:00:00,{},False,2019-07-03T16:18:39,False,2019-10-08T20:30:00,False,2019-10-09T01:30:00,,...,0.40,LUTHI,luthi,{'event_count': 5},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/luthi-tickets,2000000,concert,None
2,None,2019-08-06T00:00:00,{},False,2019-08-06T16:02:50,False,2019-10-08T19:30:00,False,2019-10-09T01:30:00,,...,0.46,Dean Z: The Ultimate Elvis,dean-z-the-ultimate-elvis,{'event_count': 1},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/dean-z-the-ultimate-elvis...,2000000,concert,None
3,None,2019-01-12T00:00:00,{},False,2019-01-12T16:08:13,False,2019-10-08T19:30:00,False,2019-10-09T01:30:00,,...,0.38,When You Wish,when-you-wish,{'event_count': 24},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_family,https://seatgeek.com/when-you-wish-tickets,3000000,theater,None
4,None,2019-02-03T00:00:00,{'checkout_disclosures': {'messages': [{'text'...,False,2019-02-03T17:24:24,False,2019-10-08T18:30:00,False,2019-10-09T01:30:00,,...,0.42,Marriage Can Be Murder,marriage-can-be-murder,{'event_count': 159},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_comedy,https://seatgeek.com/marriage-can-be-murder-ti...,3000000,theater,None
5,None,2019-08-21T00:00:00,{'checkout_disclosures': {'messages': [{'text'...,False,2019-08-21T16:13:00,False,2019-10-08T21:45:00,False,2019-10-09T01:45:00,,...,0.32,Comedy Showcase,comedy-showcase,{'event_count': 166},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_comedy,https://seatgeek.com/comedy-showcase-tickets,3000000,theater,None
6,None,2019-05-01T00:00:00,{},False,2019-05-01T16:16:31,False,2019-10-08T19:00:00,False,2019-10-09T02:00:00,,...,0.45,Roseanne Barr,roseanne-barr,{'event_count': 2},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_comedy,https://seatgeek.com/roseanne-barr-tickets,3000000,theater,None
7,None,2019-09-20T00:00:00,{},False,2019-09-20T16:49:37,False,2019-10-08T19:00:00,False,2019-10-09T02:00:00,,...,0.00,Top Girls,top-girls,{'event_count': 18},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_broadway_national_tours,https://seatgeek.com/top-girls-tickets,3000000,theater,None
8,None,2019-08-06T00:00:00,{},False,2019-08-06T16:02:52,False,2019-10-08T20:00:00,False,2019-10-09T02:00:00,,...,0.43,Jidenna,jidenna,{'event_count': 28},"[{'name': 'concerts', 'parent_id': None, 'id':...",band,https://seatgeek.com/jidenna-tickets,2000000,concert,None
9,None,2019-10-03T00:00:00,{},False,2019-10-03T16:10:08,False,2019-10-08T19:00:00,False,2019-10-09T02:00:00,,...,0.37,Troubled Comedy,troubled-comedy,{'event_count': 1},"[{'name': 'theater', 'parent_id': None, 'id': ...",theater_comedy,https://seatgeek.com/troubled-comedy-tickets,3000000,theater,None


In [74]:
# test_files = ['1.json', '2.json', '3.json', '4.json', '5.json', '6.json', '7.json', '8.json']

# result_test_df = pd.DataFrame()


# for i in test_files:
#     print("Reading in {}/{}".format(i, len(test_files)))
#     # clear the output to make things neater
#     clear_output(wait = True)
    
#     result_test_df = result_test_df.append(format_ticket_json(i))

# result_test_df

In [75]:
files = glob.glob('*.json')
files

['86166.json',
 '57896.json',
 '66332.json',
 '30552.json',
 '37297.json',
 '3721.json',
 '51041.json',
 '2833.json',
 '29289.json',
 '56784.json',
 '108928.json',
 '49888.json',
 '98178.json',
 '23684.json',
 '90548.json',
 '72721.json',
 '24141.json',
 '92575.json',
 '59471.json',
 '73833.json',
 '103237.json',
 '42397.json',
 '89293.json',
 '13232.json',
 '45452.json',
 '22996.json',
 '38162.json',
 '106035.json',
 '47195.json',
 '96865.json',
 '89769.json',
 '729.json',
 '16030.json',
 '40650.json',
 '104008.json',
 '38498.json',
 '94858.json',
 '41942.json',
 '26486.json',
 '97777.json',
 '77523.json',
 '21343.json',
 '54243.json',
 '29773.json',
 '98482.json',
 '112426.json',
 '53586.json',
 '34842.json',
 '63130.json',
 '35750.json',
 '48260.json',
 '83364.json',
 '28861.json',
 '32095.json',
 '6523.json',
 '81359.json',
 '51411.json',
 '67870.json',
 '98528.json',
 '30102.json',
 '66762.json',
 '86536.json',
 '6489.json',
 '3371.json',
 '103667.json',
 '683.json',
 '69497.json'

In [76]:
# get files that have contents beyond meta data
file_info = dict()

nonblank = list()

for file in files:
    if os.path.getsize(file) > 106:
        nonblank.append(file)

In [77]:
len(nonblank)

11594

In [78]:
nonblank_sorted = sorted(nonblank,key=lambda x: int(os.path.splitext(x)[0]))

In [79]:
# result_df = pd.DataFrame()
# errors = pd.DataFrame()

# for i in files:
#     print("Reading in {}/{}".format(i, len(files)))
# #     # clear the output to make things neater
# #     clear_output(wait = True)
    
#     result_df = result_df.append(format_ticket_json(i))


In [80]:
test_files = ['5.json', '6.json', '7.json', '1.json', '2.json', '3.json', '4.json', '8.json', '29289.json']
test_files_sorted = sorted(test_files,key=lambda x: int(os.path.splitext(x)[0]))

result_test_df = pd.DataFrame()

for i in test_files_sorted:
    print("Reading in file {}/{}".format(i, len(test_files)))
    
    result_test_df = result_test_df.append(format_ticket_json(i))

Reading in file 1.json/9
Reading in file 2.json/9
Reading in file 3.json/9
Reading in file 4.json/9
Reading in file 5.json/9
Reading in file 6.json/9
Reading in file 7.json/9
Reading in file 8.json/9
Reading in file 29289.json/9
29289.json doesn't have appropriate columns


In [83]:
result_test_df

,access_method,announce_date,announcements,conditional,created_at,date_tbd,datetime_local,datetime_tbd,datetime_utc,description,...,venue_name_v2,venue_num_upcoming_events,venue_popularity,venue_postal_code,venue_score,venue_slug,venue_state,venue_timezone,venue_url,visible_until_utc
0,None,2019-01-23T00:00:00,{'checkout_disclosures': {'messages': [{'text'...,False,2019-01-23T16:10:16,False,2019-10-08T03:30:00,False,2019-10-08T10:30:00,,...,Le Theatre Des Arts at Paris Las Vegas,161,0,89109,0.453888,le-theatre-des-arts-at-paris-las-vegas,NV,America/Los_Angeles,https://seatgeek.com/venues/le-theatre-des-art...,2019-10-09T07:00:00
1,None,2018-12-20T00:00:00,{'checkout_disclosures': {'messages': [{'text'...,False,2018-12-20T16:11:07,False,2019-10-08T03:30:00,False,2019-10-08T10:30:00,,...,High Roller Wheel At the LINQ,158,0,89109,0.000000,high-roller-wheel-at-the-linq,NV,America/Los_Angeles,https://seatgeek.com/venues/high-roller-wheel-...,2019-10-09T07:00:00
2,None,2019-05-02T00:00:00,{},False,2019-05-02T16:16:35,False,2019-10-08T19:30:00,False,2019-10-09T01:30:00,,...,Colonial Theater,1,0,83402,0.394669,colonial-theater-2,ID,America/Boise,https://seatgeek.com/venues/colonial-theater-2...,2019-10-09T05:30:00
3,None,2019-05-14T00:00:00,{},False,2019-05-14T02:59:07,False,2019-10-08T18:30:00,False,2019-10-09T01:30:00,,...,House of Blues - Anaheim,52,0,92802,0.544078,house-of-blues-anaheim,CA,America/Los_Angeles,https://seatgeek.com/venues/house-of-blues-ana...,2019-10-09T05:30:00
4,None,2019-08-24T00:00:00,{},False,2019-08-24T16:02:30,False,2019-10-08T20:30:00,False,2019-10-09T01:30:00,,...,Gasa Gasa,30,0,70115,0.354410,gasa-gasa,LA,America/Chicago,https://seatgeek.com/venues/gasa-gasa/tickets,2019-10-09T05:30:00
5,None,2019-07-13T00:00:00,{},False,2019-07-13T16:04:06,False,2019-10-08T19:30:00,False,2019-10-09T01:30:00,,...,The Oriental Theater,19,0,80212,0.322193,the-oriental-theater,CO,America/Denver,https://seatgeek.com/venues/the-oriental-theat...,2019-10-09T05:30:00
6,None,2019-09-09T00:00:00,{},False,2019-09-09T16:04:28,False,2019-10-08T20:30:00,False,2019-10-09T01:30:00,,...,Constellation,18,0,60618,0.000000,constellation,IL,America/Chicago,https://seatgeek.com/venues/constellation/tickets,2019-10-09T05:30:00
7,"{'employee_only': False, 'created_at': '2015-1...",2019-01-31T00:00:00,{},False,2019-01-31T16:17:36,False,2019-10-08T19:30:00,False,2019-10-09T01:30:00,,...,Red Rocks Amphitheatre,25,0,80465,0.745735,red-rocks-amphitheatre,CO,America/Denver,https://seatgeek.com/venues/red-rocks-amphithe...,2019-10-09T05:30:00
8,None,2019-04-26T00:00:00,{},False,2019-04-26T16:21:18,False,2019-10-08T21:30:00,False,2019-10-09T01:30:00,,...,Bar Le Ritz PDB,35,0,H2R 2X2,0.392210,bar-le-ritz-pdb-1,QC,America/Toronto,https://seatgeek.com/venues/bar-le-ritz-pdb-1/...,2019-10-09T05:30:00
9,None,2019-09-13T00:00:00,{},False,2019-09-13T16:08:37,False,2019-10-08T20:30:00,False,2019-10-09T01:30:00,,...,ColdTowne Theater,11,0,78751,0.000000,coldtowne-theater,TX,America/Chicago,https://seatgeek.com/venues/coldtowne-theater/...,2019-10-09T05:30:00


In [84]:
result_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 0 to 9
Data columns (total 88 columns):
access_method                            7 non-null object
announce_date                            80 non-null object
announcements                            80 non-null object
conditional                              80 non-null bool
created_at                               80 non-null object
date_tbd                                 80 non-null bool
datetime_local                           80 non-null object
datetime_tbd                             80 non-null bool
datetime_utc                             80 non-null object
description                              80 non-null object
enddatetime_utc                          0 non-null object
event_promotion                          0 non-null object
general_admission                        25 non-null object
id                                       80 non-null int64
is_open                                  80 non-null bool
links     

In [85]:
result_test_df['performers_name']

0                              Eiffel Tower Experience
1                                    High Roller Wheel
2                                            Swan Lake
3                                    Angels & Airwaves
4                                        The Aquadolls
5                                          Senses Fail
6                                        William Tyler
7                                      Vampire Weekend
8                                             Bedouine
9                                          A Class Act
0    Therapeutic Laughter Comedy Show[Hecklers Edit...
1                                                LUTHI
2                           Dean Z: The Ultimate Elvis
3                                        When You Wish
4                               Marriage Can Be Murder
5                                      Comedy Showcase
6                                        Roseanne Barr
7                                            Top Girls
8         

In [86]:
result_test_df.to_csv('test_result.csv')

In [88]:
result_test_df['taxonomies_name']

0    theater
1    theater
2    theater
3    concert
4    concert
5    concert
6    concert
7    concert
8    theater
9    theater
0    theater
1    concert
2    concert
3    theater
4    theater
5    theater
6    theater
7    theater
8    concert
9    theater
0    concert
1    theater
2    theater
3    theater
4     sports
5    theater
6    theater
7    concert
8    theater
9    concert
      ...   
0    concert
1     sports
2    concert
3    concert
4    concert
5    concert
6    concert
7    concert
8    theater
9    theater
0    concert
1    concert
2    concert
3    concert
4    concert
5    concert
6    theater
7    concert
8    concert
9    theater
0    concert
1    concert
2    theater
3    concert
4    concert
5    concert
6    concert
7    concert
8    concert
9    concert
Name: taxonomies_name, Length: 80, dtype: object

In [91]:
result_test_df[result_test_df['taxonomies_name'] == 'concert']['performers_name']

3                                Angels & Airwaves
4                                    The Aquadolls
5                                      Senses Fail
6                                    William Tyler
7                                  Vampire Weekend
1                                            LUTHI
2                       Dean Z: The Ultimate Elvis
8                                          Jidenna
0                                         Bleached
7                                     Mike Doughty
9                             Cigarettes After Sex
0                                     Ruston Kelly
1                  Frank Carter & The Rattlesnakes
2                                     Joshua Radin
5                                      Clint Black
7                                           Sum 41
8                  The Songbooks of SADE & MAXWELL
9    Coast 2 Coast LIVE Artist Showcase Memphis TN
0                                       Futuristic
1                              